# Assignment Part 3b: Debug Lab Results Analysis

This notebook analyzes patient glucose test results to identify diabetes risk. It contains hidden issues you need to uncover using VS Code's notebook debugger.

**Your task:**

- Use the debug icon to run cells interactively
- Set breakpoints and inspect variables
- Fix the issues and add concise comments explaining each change
- Restart the kernel + Run All to verify everything works

---

## Setup and load data

In [34]:
import pandas as pd
from pathlib import Path

# Load patient data
data_path = Path("data/patient_intake.csv")
patients = pd.read_csv(data_path)

print(f"Loaded {len(patients)} patients")
patients.head()

Loaded 50 patients


,patient_id,first_name,last_name,weight_kg,height_cm,age,sex
0,P001,Mark,Johnson,91.5,177,46,M
1,P002,Donald,Walker,80.5,164,29,M
2,P003,Nancy,Rhodes,74.3,163,47,F
3,P004,Steven,Miller,64.4,171,71,M
4,P005,Javier,Johnson,72.8,178,18,M


---

## Calculate fasting glucose estimates

In [35]:
print("Estimating fasting glucose from BMI and age...")

# Simple glucose estimation for demonstration
# (In reality, would come from lab tests)
patients["glucose_mg_dl"] = (patients["weight_kg"] * 1.2 + patients["age"] * 0.3).round(1)



# Convert to string for "display formatting"
patients["glucose_display"] = patients["glucose_mg_dl"].map(lambda x: f"{x:.1f}")

print(f"Glucose calculated for {len(patients)} patients")
print(f"Sample values:")
print(patients[["patient_id", "glucose_mg_dl"]].head())



Estimating fasting glucose from BMI and age...
Glucose calculated for 50 patients
Sample values:
  patient_id  glucose_mg_dl
0       P001          123.6
1       P002          105.3
2       P003          103.3
3       P004           98.6
4       P005           92.8


---

## Categorize diabetes risk

In [36]:
print("\nCategorizing diabetes risk based on fasting glucose...")

def categorize_glucose(glucose_value):
    """Categorize diabetes risk from fasting glucose (mg/dL)."""
    if glucose_value < 100:
        return "Low risk (normal)"
    elif glucose_value < 126:
        return "High risk (prediabetes)"
    else:
        return "Very high risk (diabetes)"

patients["glucose_mg_dl"] = pd.to_numeric(patients["glucose_mg_dl"], errors="coerce")

patients["diabetes_risk"] = patients["glucose_mg_dl"].apply(categorize_glucose)

print("Risk categories assigned:")
print(patients[["patient_id", "glucose_mg_dl", "diabetes_risk"]].head(10))
print(patients["diabetes_risk"].value_counts())


Categorizing diabetes risk based on fasting glucose...
Risk categories assigned:
  patient_id  glucose_mg_dl            diabetes_risk
0       P001          123.6  High risk (prediabetes)
1       P002          105.3  High risk (prediabetes)
2       P003          103.3  High risk (prediabetes)
3       P004           98.6        Low risk (normal)
4       P005           92.8        Low risk (normal)
5       P006          105.3  High risk (prediabetes)
6       P007          119.7  High risk (prediabetes)
7       P008          118.4  High risk (prediabetes)
8       P009          115.9  High risk (prediabetes)
9       P010           91.9        Low risk (normal)
diabetes_risk
High risk (prediabetes)      33
Low risk (normal)            11
Very high risk (diabetes)     6
Name: count, dtype: int64


---

## Filter high-risk patients

In [40]:
print("\nIdentifying patients needing follow-up...")

# Find patients with elevated glucose
high_risk = patients[
    patients["diabetes_risk"].str.contains("high risk", case=False, na=False)].copy()

print(f"Found {len(high_risk)} patients with elevated glucose")
if len(high_risk) > 0:
    print(f"Glucose range: {high_risk['glucose_mg_dl'].min()} to {high_risk['glucose_mg_dl'].max()}")


Identifying patients needing follow-up...
Found 39 patients with elevated glucose
Glucose range: 100.2 to 141.9


---

## Calculate intervention priority scores

In [41]:
print("\nCalculating intervention priority scores...")

priority_patients = []
records = high_risk.to_dict("records")

# Calculate priority score for each high-risk patient
for i in range(len(records)):
    patient = records[i]

    # Priority score: higher glucose + older age = higher priority
    glucose = float(patient["glucose_mg_dl"])
    age = int(patient["age"])
    priority_score = (glucose - 100) + (age * 0.5)

    priority_patients.append({
        "patient_id": patient["patient_id"],
        "glucose": glucose,
        "age": age,
        "priority_score": round(priority_score, 1)
    })

# Sort by priority score
priority_patients.sort(key=lambda x: x["priority_score"], reverse=True)

print(f"Priority scores calculated for {len(priority_patients)} patients")
if priority_patients:
    print(f"\nTop 3 priority patients:")
    for p in priority_patients[:3]:
        print(f"  {p['patient_id']}: score {p['priority_score']} (glucose={p['glucose']}, age={p['age']})")


Calculating intervention priority scores...
Priority scores calculated for 39 patients

Top 3 priority patients:
  P031: score 80.4 (glucose=137.9, age=85)
  P046: score 68.6 (glucose=132.6, age=72)
  P029: score 67.4 (glucose=141.9, age=51)


---

## Summary

In [42]:
print("\n" + "=" * 50)
print("Lab Results Analysis Complete")
print("=" * 50)
print(f"Total patients analyzed: {len(patients)}")
print(f"High-risk patients: {len(high_risk)}")
print(f"Patients prioritized for intervention: {len(priority_patients)}")


Lab Results Analysis Complete
Total patients analyzed: 50
High-risk patients: 39
Patients prioritized for intervention: 39


---

## Debugging Checklist

After fixing all bugs, verify:

- [ ] Runs without errors end-to-end
- [ ] Glucose values are reasonable numbers
- [ ] Risk categories make sense (high glucose = high risk)
- [ ] All high-risk patients are identified
- [ ] Priority scores calculated correctly
- [ ] Restart kernel + Run All completes successfully
- [ ] Added comments explaining each fix